# Preprocessing
讀取資料

In [1]:
import pandas as pd
import numpy as np

In [2]:
# path = "/content/drive/MyDrive/"
# path = ''
path = '/kaggle/input/'

train_df = pd.read_csv(path + "store-sales-time-series-forecasting/train.csv")
holiday_event_df = pd.read_csv(path + "store-sales-time-series-forecasting/holidays_events.csv")
stores_df = pd.read_csv(path + "store-sales-time-series-forecasting/stores.csv")
# transaction_df = pd.read_csv(path + "store-sales-time-series-forecasting/transactions.csv")
oil_df = pd.read_csv(path + "store-sales-time-series-forecasting/oil.csv")
test_df = pd.read_csv(path + "store-sales-time-series-forecasting/test.csv")

擷取2015/08/15~2017/08/15的資料（共兩年）

In [3]:
train_df['date'] = pd.to_datetime(train_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
holiday_event_df['date'] = pd.to_datetime(holiday_event_df['date'])
# transaction_df['date'] = pd.to_datetime(transaction_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])
# reindex oil data
oil_df = oil_df.merge(
    pd.DataFrame({"date": pd.date_range(train_df.date.min().date(), test_df.date.max().date())}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

# fill missing values using linear interpolation
oil_df.dcoilwtico = oil_df.dcoilwtico.interpolate(method="linear", limit_direction="both")

train_df = train_df[(train_df['date'] >= '2014-08-15') & (train_df['date'] <= '2017-12-31')]
holiday_event_df = holiday_event_df[(holiday_event_df['date'] >= '2014-08-15') & (holiday_event_df['date'] <= '2017-12-31')]
oil_df = oil_df[(oil_df['date'] >= '2014-08-15') & (oil_df['date'] <= '2017-12-31')]
# transaction_df = transaction_df[(transaction_df['date'] >= '2015-08-15') & (transaction_df['date'] <= '2017-12-31')]
test_df = test_df[(test_df['date'] >= '2014-08-15') & (test_df['date'] <= '2017-12-31')]

合併資料

In [4]:
print(len(train_df))
print(len(test_df))
train_df = pd.merge(train_df, oil_df, on='date', how='left')
train_df = pd.merge(train_df, stores_df, on='store_nbr', how='left')
# train_df = pd.merge(train_df, transaction_df, on=['date', 'store_nbr'], how='left')
test_df = pd.merge(test_df, oil_df, on='date', how='left')
test_df = pd.merge(test_df, stores_df, on='store_nbr', how='left')
print(train_df['id'].nunique())

del oil_df, stores_df

1949508
28512
1949508


In [5]:
national_df = holiday_event_df[holiday_event_df['locale'] == 'National']
national_df = national_df.drop_duplicates(subset='date', keep='first')
train_national_df = pd.merge(train_df, national_df, on='date', how='left')
non_national_df = holiday_event_df[holiday_event_df['locale'] != 'National']
non_national_df = non_national_df.drop_duplicates(subset=['date', 'locale_name'], keep='first')
train_non_national_df = pd.merge(train_df, non_national_df, left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')
train_df = train_national_df.combine_first(train_non_national_df)

test_national_df = pd.merge(test_df, national_df, on='date', how='left')
test_non_national_df = pd.merge(test_df, non_national_df, left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left')
test_df = test_national_df.combine_first(test_non_national_df)

del national_df, non_national_df, train_national_df, train_non_national_df, test_national_df, test_non_national_df

欄位名稱轉換（因為原始資料集有兩個type欄位分別在holiday_event_df跟stores_df）

---



In [6]:
train_df = train_df.rename(columns={'type_x': 'store_type', 'type_y': 'event_type'})
test_df = test_df.rename(columns={'type_x': 'store_type', 'type_y': 'event_type'})

特徵工程（1. 日期欄位拆分、2. 數值欄位 min-max normalization）

In [7]:
# 日期欄位拆分
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['day_of_week'] = train_df['date'].dt.dayofweek

test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['day_of_week'] = test_df['date'].dt.dayofweek

In [8]:
# 數值欄位 min-max normalization
columns_to_normalize = ['onpromotion', 'dcoilwtico']
train_df[columns_to_normalize] = (train_df[columns_to_normalize] - train_df[columns_to_normalize].min()) / (train_df[columns_to_normalize].max() - train_df[columns_to_normalize].min())
test_df[columns_to_normalize] = (test_df[columns_to_normalize] - train_df[columns_to_normalize].min()) / (test_df[columns_to_normalize].max() - train_df[columns_to_normalize].min())

新增經緯度欄位

In [ ]:
cities_coordinates = {
    "Quito": {"lat": -0.1806532, "lon": -78.4678382},
    "Guayaquil": {"lat": -2.1709979, "lon": -79.9223592},
    "Santo Domingo": {"lat": -0.2530494, "lon": -79.1753765},
    "Cuenca": {"lat": -2.9001285, "lon": -79.0058965},
    "Latacunga": {"lat": -0.9393387, "lon": -78.6155545},
    "Manta": {"lat": -0.9676533, "lon": -80.7089101},
    "Machala": {"lat": -3.2581112, "lon": -79.9553924},
    "Ambato": {"lat": -1.2416666, "lon": -78.6195459},
    "Quevedo": {"lat": -1.0225124, "lon": -79.4604035},
    "Esmeraldas": {"lat": 0.9681789, "lon": -79.6517202},
    "Loja": {"lat": -3.9931283, "lon": -79.2042216},
    "Libertad": {"lat": -2.2311612, "lon": -80.9008852},
    "Playas": {"lat": -2.6284683, "lon": -80.3895886},
    "Daule": {"lat": -1.8621807, "lon": -79.9776688},
    "Babahoyo": {"lat": -1.8019264, "lon": -79.5346458},
    "Cayambe": {"lat": 0.0430556, "lon": -78.1459943},
    "Salinas": {"lat": -2.2171001, "lon": -80.9586051},
    "Puyo": {"lat": -1.4923925, "lon": -78.0024134},
    "Guaranda": {"lat": -1.5904732, "lon": -79.0022925},
    "Ibarra": {"lat": 0.3517083, "lon": -78.1223373},
    "Riobamba": {"lat": -1.6635508, "lon": -78.654646},
    "El Carmen": {"lat": -0.2687816, "lon": -79.466199}
}

train_df['coordinates'] = train_df['city'].map(cities_coordinates)
test_df['coordinates'] = test_df['city'].map(cities_coordinates)

train_df['longitude'] = train_df['coordinates'].map(lambda x: x['lon'] if x is not None else None)
train_df['latitude'] = train_df['coordinates'].map(lambda x: x['lat'] if x is not None else None)
test_df['longitude'] = test_df['coordinates'].map(lambda x: x['lon'] if x is not None else None)
test_df['latitude'] = test_df['coordinates'].map(lambda x: x['lat'] if x is not None else None)

train_df.drop('coordinates', axis=1, inplace=True)
test_df.drop('coordinates', axis=1, inplace=True)

# convert day_of_week to sin and cos
train_df['day_of_week_sin'] = np.sin(2 * np.pi * train_df['day_of_week']/7.0)
train_df['day_of_week_cos'] = np.cos(2 * np.pi * train_df['day_of_week']/7.0)
test_df['day_of_week_sin'] = np.sin(2 * np.pi * test_df['day_of_week']/7.0)
test_df['day_of_week_cos'] = np.cos(2 * np.pi * test_df['day_of_week']/7.0)


# Use one-hot encoder
from sklearn.preprocessing import OneHotEncoder

one_hot_col = ['store_type']

# Instantiate and fit the OneHotEncoder
enc = OneHotEncoder(sparse=False)
enc.fit(train_df[one_hot_col])

# Transform the data
train_one_hot = enc.transform(train_df[one_hot_col])
test_one_hot = enc.transform(test_df[one_hot_col])

# Get the feature names
feature_names = enc.get_feature_names_out(one_hot_col)

# Create DataFrames from the one-hot encoded arrays
train_one_hot_df = pd.DataFrame(train_one_hot, columns=feature_names, index=train_df.index)
test_one_hot_df = pd.DataFrame(test_one_hot, columns=feature_names, index=test_df.index)


# Concatenate the one-hot encoded columns to the original DataFrames
train_df = pd.concat([train_df, train_one_hot_df], axis=1)
test_df = pd.concat([test_df, test_one_hot_df], axis=1)

# Drop the original columns
train_df.drop(one_hot_col, axis=1, inplace=True)
test_df.drop(one_hot_col, axis=1, inplace=True)

del train_one_hot_df, test_one_hot_df, feature_names, enc
# Display the DataFrames to check the results
# print(train_df.head())
# print(test_df.head())


def replace_transfer(row):
    if row['event_type'] == 'Holiday':
        if row['transferred'] == 1:
            return 0
        return 1
    elif row['event_type'] == 'Transfer':
        return 1
    elif row['event_type'] == 'Additional':
        return 1
    elif row['event_type'] == 'Bridge':
        return 1
    return 0

def isEvent(row):
    if row['event_type'] == 'Event':
        return 1
    return 0

train_df['isHoliday'] = train_df.apply(replace_transfer, axis=1)
train_df['isEvent'] = train_df.apply(isEvent, axis=1)
test_df['isHoliday'] = test_df.apply(replace_transfer, axis=1)
test_df['isEvent'] = test_df.apply(isEvent, axis=1)

train_df['dcoilwtico'].fillna(train_df['dcoilwtico'].mean(), inplace=True)
test_df['dcoilwtico'].fillna(train_df['dcoilwtico'].mean(), inplace=True)
train_df = train_df.drop(['description', 'city', 'state', 'cluster', 'year', 'event_type', 'locale', 'locale_name', 'transferred'], axis=1)
test_df = test_df.drop(['description', 'city', 'state', 'cluster', 'year', 'event_type', 'locale', 'locale_name', 'transferred'], axis=1)

把sales欄位放到資料集最後，作為y_train

In [12]:
cols = [col for col in train_df.columns if col != 'sales']
cols.append('sales')
train_df = train_df[cols]
# test_df = test_df[cols]

# Transformation
## Multi-Index

In [14]:
# conduct multiindex
train_df['date'] = train_df.date.dt.to_period('D')
test_df['date'] = test_df.date.dt.to_period('D')

# Set the new index
train_df = (train_df
       .set_index(['date', 'family', 'store_nbr'])         # Setting MultiIndex to make unique identifiers for each 'sales' item
       .sort_index()
      )

test_df = (test_df
       .set_index(['date', 'family', 'store_nbr'])         # Setting MultiIndex to make unique identifiers for each 'sales' item
       .sort_index()
      )

In [15]:
X_train = train_df.copy()
X_test = test_df.copy()

del train_df, test_df
# Pivot the table to get the desired structure
y_train = (X_train
           .unstack(['family', 'store_nbr'])
           .loc[:,"sales"]
          )

X_train = (X_train
           .unstack(['family', 'store_nbr'])
           .loc[:, X_train.columns[0]:X_train.columns[-1]]
          )
# print(X_train.shape)

X_test = (X_test
          .unstack(['family', 'store_nbr'])
          .loc[:, X_test.columns[0]:X_test.columns[-1]]
         )
# get X_test have same columns as X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

X_train_id = X_train['id']
X_test_id = X_test['id']

X_train = X_train.drop('id', axis=1)
X_test = X_test.drop('id', axis=1)

# drop duplicated columns
X_train_dup_columns = ~X_train.columns.duplicated()
X_train = X_train.loc[:,X_train_dup_columns]
X_test = X_test.loc[:,X_train_dup_columns]

print(X_train.shape)
print(X_test.shape)

(1094, 30294)
(16, 30294)


## Single-Index

In [16]:
# # split data by store_nbr and family
# store_nbrs = train_df['store_nbr'].unique()
# families = train_df['family'].unique()
# train_famlies_df = []
# train_stores_df = []
# test_famlies_df = []
# test_stores_df = []
# test_id_df = []
# for i, family in enumerate(families):
#     train_famlies_df.append([])
#     test_famlies_df.append([])
#     test_id_df.append([])
#     for j, store_nbr in enumerate(store_nbrs):
#         df = train_df[(train_df['store_nbr'] == store_nbr) & (train_df['family'] == family)]
#         df_test = test_df[(test_df['store_nbr'] == store_nbr) & (test_df['family'] == family)]
#         # sort by date
#         df = df.sort_values(by='date')
#         df_test = df_test.sort_values(by='date')

#         test_id_df[i].append(df_test[['id']])
#         # drop date column
#         df = df.drop(['date', 'id', 'family', 'store_nbr'], axis=1)
#         df_test = df_test.drop(['date', 'id', 'family', 'store_nbr'], axis=1)
        
#         if len(df) > 0:
#             train_famlies_df[i].append(df)
#         if len(df_test) > 0:
#             test_famlies_df[i].append(df_test)

# LSTM Model
## Multi-Index

In [ ]:
#### construct LSTM model with keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate


def create_sequences(data_X, data_Y, time_steps=1):
    x, y = [], []
    
    for i in range(len(data_X) - time_steps + 1):
        x.append(data_X.iloc[i:(i + time_steps)].values)
        y.append(data_Y.iloc[i + time_steps -1])
    return np.array(x), np.array(y)

def create_model(input_shape, lstm_units=1782, dense_units=50, dropout_rate=0.2):
    input_layer = Input(shape=input_shape)
    # add multiple LSTM layers and dropout layers
    lstm_layer = LSTM(lstm_units, return_sequences=True)(input_layer)
    lstm_layer = Dropout(dropout_rate)(lstm_layer)
    lstm_layer = LSTM(lstm_units)(lstm_layer)
    lstm_layer = Dropout(dropout_rate)(lstm_layer)
    # add dense layer
    output_layer = Dense(1782, activation='relu')(lstm_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    # compile with MSLE
    model.compile(optimizer='adam', loss='mean_squared_logarithmic_error')
    # show model summary
    model.summary()
    return model

time_steps = 32
batch_size = 4
X_test.fillna(0, inplace=True)
complete_df = pd.concat([X_train, X_test], axis=0)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
# print(y_train_seq[0]
df_test = complete_df[len(X_train)-time_steps + 1:]
del complete_df, X_train
X_test_seq, y_test_seq = create_sequences(df_test, y_train, time_steps)
del df_test
print(X_train_seq.shape)
print(y_train_seq.shape)
model = create_model((X_train_seq.shape[1], X_train_seq.shape[2]))
model.fit(X_train_seq, y_train_seq, epochs=30, batch_size=batch_size, verbose=1)
y_pred = model.predict(X_test_seq, verbose=1)

2024-06-13 02:29:29.765311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 02:29:29.765478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 02:29:29.912781: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(1063, 32, 30294)
(1063, 1782)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 30294)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32, 1782)       │   228,644,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 1782)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1782)           │    25,411,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1782)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1782)           │     3,177,306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257,233,482 (981.27 MB)

 Trainable params: 257,233,482 (981.27 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30


## Single-Index

In [ ]:
# # construct LSTM model with keras
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# from keras.layers import Embedding
# from keras.layers import Input
# from keras.models import Model
# from keras.layers import concatenate


# def create_sequences(data_X, data_Y, time_steps=1):
#     x, y = [], []
#     for i in range(len(data_X) - time_steps + 1):
#         x.append(data_X.iloc[i:(i + time_steps)].values)
#         y.append(data_Y.iloc[i + time_steps -1])
#     return np.array(x), np.array(y)

# def create_model(input_shape, lstm_units=50, dense_units=50, dropout_rate=0.2):
#     input_layer = Input(shape=input_shape)
#     lstm_layer = LSTM(lstm_units)(input_layer)
#     output_layer = Dense(1)(lstm_layer)
#     model = Model(inputs=input_layer, outputs=output_layer)
#     model.compile(optimizer='adam', loss='mse')
#     return model

# time_steps = 1
# batch_size = 128

# for i in range(len(train_famlies_df)):
#     for j in range(len(train_famlies_df[i])):
#         df = train_famlies_df[i][j]
#         df_test = test_famlies_df[i][j]
#         if len(df) == 0 or len(df_test) == 0:
#             continue
#         complete_df = pd.concat([df, df_test])
#         X_train, y_train = create_sequences(df.drop('sales', axis=1), df[['sales']], time_steps)
#         df_test = complete_df[len(df)-time_steps + 1:]
#         X_test, y_test = create_sequences(df_test.drop('sales', axis=1), df_test[['sales']], time_steps)
#         model = create_model((X_train.shape[1], X_train.shape[2]))
#         model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=0)
#         y_pred = model.predict(X_test, verbose=0)
#         test_id_df[i][j]['sales'] = y_pred
#         del model, X_train, y_train, X_test, y_test, complete_df, df, df_test, y_pred
#     print(i)

# Submission
## Single-Index

In [ ]:
# # save prediction csv 
# test_id_df_combined = pd.DataFrame()
# for i in range(len(test_id_df)):
#     for j in range(len(test_id_df[i])):
#         test_id_df_combined = pd.concat([test_id_df_combined, test_id_df[i][j]])

# # merge test_id_df_combined with test_df
# test_df_result = pd.merge(test_df, test_id_df_combined, on='id', how='left')
# print(test_df_result)

# test_df_result[['id', 'sales']].to_csv('/kaggle/working/submission.csv', index=False)

## Multi-Index

In [ ]:
import pandas as pd

# Assuming y_pred is the predictions with shape (16, 1782)
# and X_test_id contains the original IDs corresponding to the same multi-indexed structure.

# Step 1: Reshape y_pred to match the structure of test_df before MultiIndex
y_pred_df = pd.DataFrame(y_pred, index=X_test.index, columns=y_train.columns)

# Step 2: Use X_test_id to map the predictions back to their original IDs
# Create a DataFrame to hold the IDs and predictions
predictions_df = y_pred_df.stack(['family', 'store_nbr']).reset_index()
predictions_df.columns = ['date', 'family', 'store_nbr', 'sales']

# Add the original IDs
predictions_df['id'] = X_test_id.stack(['family', 'store_nbr']).values

# Step 3: Reconstruct the test_df DataFrame using the original IDs and the predictions
# Assuming the original test_df had the columns ['id', 'date', 'family', 'store_nbr', 'sales']
final_df = predictions_df[['id', 'date', 'family', 'store_nbr', 'sales']]

# Sort by 'id' if necessary
final_df = final_df.sort_values(by='id').reset_index(drop=True)

# Display the final DataFrame
# print(final_df)

submission_df = final_df[['id', 'sales']]
# Save to CSV if required
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print(submission_df)